# Projeto 1 - Ciência dos Dados

Nome: Giovanni Rozatti

Nome: Gabriel Parfan Guimarães

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

___
## Classificador automático de sentimento


- Produto escolhido: Atila Iamariano
- Descrição do Produto: 

    Atila Iamarino (1984)é um biólogo e pesquisador brasileiro formado em microbiologia, possuindo também doutorado em virologia, notório por seu trabalho no canal de YouTube denominado "Nerdologia",que possui mais de 2 milhões de inscritos e integra o grupo Jovem Nerd, uma plataforma digital de cultura pop, jogos eletrônicos e conteúdo jovem. -Wikipedia  
    
    Átila já era bastante conhecido por seu canal no Youtube e por sua participação em podcasts, principalment do podcast Jovem Nerd) popularidade de Atila cresceu bastante atualmente, principalmente por conta do COVID-19, chegando a transmitir lives diárias, para falar da situação atual, cuidados preventivos e fazendo predições com uso de estatísticas e dados sobre como a situação se desdobraria, chegando a até mesmo a participar de um Programa do Roda-Viva `https://www.youtube.com/watch?v=s00BzYazxvU` - Observação de Giovanni Rozatti
    ________________________________________________________________________________________________

- Método de classificação: 
foram realizados 2 níveis de classificação, no primeiro foi classificada a relêvancia da mensagem, e no segundo era avaliado o teor da mensagem
___

Nível 1 de classificação:

- Relevante: Tweets que contribuiriam com o processo de classificação por estarem relacionados ou ao tema discutido ou a algo próximo e/ou que continham elementos possiveis para que fosse feita a classificação do teor de sua mensagem  
    
- Irrelevante: Tweets que não contribuiam com o processo de classificação e/ou não podiam ser classificados por falta de elementos que possibilitassem isso,como por exemplo: "@atila que hrs são" (esse tweet de fato apareeu na base de dados)  
    
- Opinião Política: Tweets que continham mensagens de teor político e/ou próximos a política explícitos, como por exemplo, os termos que apareceram com recorrencia considerável : "bolsominion" e "comunista"
    
Nesse nível as mensagens foram separadas nas três categorias acima, com o objetivo de fazer uma refiltragem dos tweets obtidos, originalmente seriam apenas duas categorias (Relevante e Irrelevante), no entanto ao decorrer do processo foi vista a necessidade de estender ainda mais esse escopo, isso possibilitará a classificação das mensagens excluindo as irrelevantes e as que possuem teor político, assim como a classificação excluindo apenas as irrelevantes nos fornecendo como resultado a resposta da pergunta:
"Quanto as mensagens com teor político influenciam de fato na classificação?"
___


as frases foram classificadas de acordo com a seguinte estrutura:
I)Se possível classificar como o classifiador Naive Bayes final do projeto fará, isso é analisar as palavras individualmente tentando prever o resultado da classificação.*
II)Se não possível via método anterior classificar a frase como um todo
- Classificações atribuidas e o que levou a atribuirmos fraases a elas 
    - Show and tell: Tweets que em geral além de serem mensagens de apoiar/concordar, traziam perguntas, pesquisas, dados, materiais de referencia e fontes para embasamentos e reflexões, em geral continham teor similar ao dos exemplos a seguir: "@oatila @maisalguem vocês viram isso?", "@atila, o que voce acha de...?"  
    
    - Apoia/concorda: Tweets que em geral dizem: "@oatila parabens pelo trabalho", "@oatila bacana essa info que você apresentou"  
    
    - Discorda/Desacredita: Tweets que em geral fazem críticas ao trabalho feito, ou a uma posição que foi tomada, não necessariamente apresentando embasamento sobre o porque da crítica  
    
    - Mensagem de ódio:Tweets que apresentam mensagens ofensivas a esmo, (não necessáriamente ao biologo, mas ainda assim perinentes ao trabalho como será visto mais adiante nesse Notebook)  
    
- Observação: a estrutura do código a seguir é similar ao do usado no meu trabalho no semestre anterior (perfil no git: VonIgnia), e pode ser visto em: `https://github.com/carolinechaim/Projeto_2/blob/master/NaiveBayes teste 1.ipynb` mais especificamente nos commits "Planilha teste rodando no Jupyter" e "Planilha teste rodando os classificadores NaiveBayes".

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [ ]:
train = pd.read_excel("@oatila1.xlsx",sheet_name = 'Treinamento', index = False)
#Lê as planilhas de treinamento

In [ ]:
probs = train["Opinião Pública"].value_counts(True);
#Calculando as frequências relativas de cada categoria

In [ ]:
prob_OP = probs[0] #Opinião Positiva
prob_ON = probs[1] #Opinião Negativa

Separa todas as palavras que aparecem em cada categoria, não levando em conta repetições
(caso uma palavra apareça mais de uma vez em uma mesma categoria, ela será contabilizada apenas uma vez naquela categoria 

In [ ]:
ON = train[train["Opinião"] == 0]
ON = ON["Treinamento"].str.cat().split()
series_ON = pd.Series(ON)

In [ ]:
OP = train[train["Opinião"] == 1]
OP = OP["Treinamento"].str.cat().split()
series_OP = pd.Series(OP)

Criando uma "tabela" com a probabilidade individual de todas as palavras aparecerem em uma dada categoria
("Dado que a opinião é essa qual a probabilidade dessa palavra aparecer?")

In [ ]:
tabela_negativa = series_ON.value_counts(True)
tabela_negativa;

In [ ]:
tabela_positiva = series_OP.value_counts(True)
tabela_positiva;

Criando uma função que faz o calculo da frase pertencer a uma categoria utilizando a fórmula de bayes

In [ ]:
def classificador_de_palavras_naive_bayes(tweet):
    lista_palavras = tweet.split()
    prob_positivo = 1
    prob_negativo = 1
    for palavra in lista_palavras:
        if palavra in tabela_negativa.index:
            prob_palavra_negativa = tabela_negativa[palavra]
        else:
            prob_palavra_negativa = 1
        if palavra in tabela_positiva.index:
             prob_palavra_positivo = tabela_positiva[palavra]
        else:
             prob_palavra_positivo = 1
        prob_negativo *= prob_palavra_negativa
        prob_positivo *= prob_palavra_positivo
        if (prob_negativo > prob_positivo):
            return "Negativo"
        else:
            return "Positivo"

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

leitura da planilha para testes com o classificador

In [ ]:
test = pd.read_excel("Cinemark segunda 16-03.xlsx",sheet_name = 'Teste', index = False).set_index("Teste")
lista_tweets = list(test.index)

In [ ]:
for tweet in lista_tweets:
    test.loc[tweet,"Classificação"] = classificador_de_palavras_naive_bayes(tweet)
test

___
### Concluindo

Versão 1  (o objeto de clasificação ainda era Cinemark, mas como os erros daquela versão poderiam se repetir nessa e serviram para reflexões além de elaborações de uma maneira de "consertá-los".)
22/03- o código roda mas claramente a classificação está com imperfeições isso pode se dar por:
 - erros de contabilização na fórmula do NaiveBayes
 - erros na clasificação do excel
 - certas palavras possuirem um viés deterministico mais forte para um dos lados (no caso a Opinião Positiva)

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**